In [108]:
import pandas as pd
from datetime import datetime

In [194]:
df = pd.read_csv('/Users/kaitlinsinger/wa_notify/DEN_future/data_extraction/manipulated_data/simplified_wide_df_with_date_loc.csv')

In [195]:
#[x for x in df.columns if x.endswith('count') or x.endswith('perc')]
df.columns

Index(['Unnamed: 0', 'source', 'pm_start_date', 'pm_end_date', 'pm_location',
       'cases_assigned_count', 'cases_interviewed_count',
       'cases_interviewed_perc', 'cases_named_contacts_count',
       'cases_not_assigned_count', 'cases_per_staff_month',
       'cases_positive_from_test_mean', 'cases_positive_from_test_med',
       'cases_reached_count', 'cases_reached_from_exposure_mean',
       'cases_reached_from_positive_max', 'cases_reached_from_positive_med',
       'cases_reached_from_symptom_mean', 'cases_reached_from_symptom_med',
       'cases_reached_from_test_mean', 'cases_reached_from_test_med',
       'cases_reached_perc', 'cases_test_from_symptom_med',
       'cases_were_contacts_count', 'contacts_assigned_count',
       'contacts_infected_count', 'contacts_infected_perc',
       'contacts_interviewed_count', 'contacts_interviewed_perc',
       'contacts_named_count', 'contacts_named_count_household',
       'contacts_named_perc_household', 'contacts_named_ratio_mean

In [196]:
# will need to fix these rows manually
df[df['pm_location'].str.endswith('Connecticut')]['pm_location'] = 'Connecticut'

/var/folders/5_/xbdfsmkd6pq54320tdmh8fth0000gn/T/ipykernel_49840/2889793188.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['pm_location'].str.endswith('Connecticut')]['pm_location'] = 'Connecticut'


In [197]:
# list of cols that have both count and perc
cols_to_impute = ['cases_assigned', 'cases_interviewed', 'cases_reached',
                  'contacts_infected', 'contacts_interviewed', 'contacts_named',
                 'contacts_reached']

# TODO: ratio of reported contacts to contacts reporting cases
# add perc naming contacts

imputation_dict = {
    #'cases_assigned_count': '?', 
    #'cases_assigned_perc': 'df["cases_assigned_count"]/(df["cases_assigned_count"]+df["cases_not_assigned_count"])*100', 
    #'cases_interviewed_count': '?', 
    'cases_interviewed_perc': 'df["cases_interviewed_count"]/df["cases_assigned_count"]*100',
    #'cases_reached_count': '',
    'cases_reached_perc': 'df["cases_reached_count"]/df["cases_assigned_count"]*100',
    
    #'contacts_named_count': '',
    # take another look at this
    # percent of cases who named at least one contact - rerun but see fill of orig data
    'percent_naming_contacts': 'df["cases_named_contacts_count"]/df["cases_interviewed_count"]*100',
    #'contacts_infected_count': '', 
    # contacts_named_count or contacts_interviewed_count (which is more populated) - look at paper's definition
    'contacts_infected_perc': 'df["contacts_infected_count"]/df["contacts_named_count"]*100',
    #'contacts_interviewed_count': '', 
    'contacts_interviewed_perc': 'df["contacts_interviewed_count"]/df["contacts_named_count"]*100',
    #'contacts_reached_count': '',
    'contacts_reached_perc': 'df["contacts_reached_count"]/df["contacts_named_count"]*100',
    'contacts_named_perc_household': 'df["contacts_named_count_household"] / df["contacts_named_count"]'
}

In [198]:
# only inputes is the data is missing
df['cases_missed_count'] = None
df['cases_missed_perc'] = None
for column, equation in imputation_dict.items():
    df[column] = df.apply(
        lambda row: eval(equation, {}, {"df": row}) if pd.isna(row[column]) else row[column], 
        axis=1
    )

In [199]:
df.head()

,Unnamed: 0,source,pm_start_date,pm_end_date,pm_location,cases_assigned_count,cases_interviewed_count,cases_interviewed_perc,cases_named_contacts_count,cases_not_assigned_count,...,self_report_codes_issued,staff_hired_ci_count,staff_hired_ci_per_health_dept_mean,staff_hired_count,staff_hired_ct_count,staff_hired_ct_per_health_dept_mean,user_cases_notifying,users_with_code,cases_missed_count,cases_missed_perc
0,0,A Multifaceted Evaluation of a COVID-19 Contac...,03/01/2021,03/31/2021,King County,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1,1,A Multifaceted Evaluation of a COVID-19 Contac...,04/01/2021,06/30/2021,King County,NaN,NaN,76.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,2,A Multifaceted Evaluation of a COVID-19 Contac...,07/01/2020,06/30/2021,King County,52317.0,42900.0,82.000115,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,3,A Multifaceted Evaluation of a COVID-19 Contac...,07/01/2020,09/30/2020,King County,NaN,NaN,87.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
4,4,A Patient-Initiated Digital COVID-19 Contact N...,05/18/2020,08/17/2020,United States*,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [200]:
region = {
    "loc_A": "West",
    "loc_B": "South",
    "loc_C": "Northeast",
    "loc_D": "Midwest",
    "loc_E": "West",
    "loc_F": "Midwest",
    "loc_G": "West",
    "loc_H": "Midwest",
    "loc_I": "South",
    "loc_J": "West",
    "loc_K": "Northeast",
    "loc_L": "South",
    "loc_M": "Midwest",
    "loc_N": "South",
    "King County": "Northwest",
    "WA": "Northwest",
    "Washington": "Northwest",
    "central WA": "Northwest",
    "Maryland": "Northeast",
    "Kansas": "Midwest",
    "United States": "US",
    "across US": "US",
    "United States*": "US",
    "Philadelphia": "Midwest",
    "Maimi University": "Midwest", #OH
    "San Fransico": "Northwest", #typo?
    'San Francisco': "Northwest",
    "California": "Northwest",
    "Members of large CA health care plan": "Northwest",
    "Boston University": "Northeast",
    "Oak Ridge National Laboratory": "South", #TN
    "New York City": "Northeast",
    "NY": "Northeast",
    "New York": "Northeast",
    "New York City communities with higher than average COVID-19 incidence rates": "Northeast",
    "New York except NYC": "Northeast",
    "North Carolina": "Northeast",
    "Randolph County": "Northeast", #NC
    "Mecklenburg": "Northeast", #NC
    "64 HD accross US and collonies": "US",
    "Connecticut": "Northeast",
    '\xa0Connecticut': "Northeast",
    "New Haven": "Northeast",
    "Michigan": "Midwest",
    "Wisconsin": "Midwest",
    "Salt Lake City": "Northwest",
    "DE": "Northeast",
    "Fulton County": "South", #ATL
    "VT": "Northeast",
    "Chicago": "Midwest",
    "Maricopa": "South", #AZ
    "Alaska": "Alaska",
    "Wyoming": "West",
    "West Virginia": "South",
    "Virginia": "Northeast",
    "Vermont": "Northeast"
}

In [201]:
# https://www.cdc.gov/ncird/whats-new/covid-19-can-surge-throughout-the-year.html
time_periods = {
    "1/1/2020-5/1/2020": "T1",
    "5/2/2020-10/15/2020": "T2",
    "10/16/2020-3/1/2021": "T3",
    "3/2/2021-5/15/2021": "T4",
    "5/15/2021-9/15/2021": "T5",
    "9/16/2021-3/1/2022": "T6",
    "3/2/2022-9/15/2022": "T7",
    "9/16/2022-3/15/2023": "T8",
    "3/16/2023-9/1/2023": "T9",
    "9/2/2023-3/15/2024": "T8"
}

def assign_time_period(date):
    for start, end, period in time_ranges:
        if start <= date <= end:
            return period
    return None

time_ranges = []
for k, v in time_periods.items():
    start_str, end_str = k.split("-")
    start_dt = datetime.strptime(start_str, "%m/%d/%Y")
    end_dt = datetime.strptime(end_str, "%m/%d/%Y")
    time_ranges.append((start_dt, end_dt, v))

In [202]:
# replace pm_location with region
df['region'] = df['pm_location'].map(region)

# add time period data
df["pm_start_date"] = pd.to_datetime(df["pm_start_date"])
df["time_period"] = df["pm_start_date"].apply(assign_time_period)

In [203]:
df[df.region.isna()]

,Unnamed: 0,source,pm_start_date,pm_end_date,pm_location,cases_assigned_count,cases_interviewed_count,cases_interviewed_perc,cases_named_contacts_count,cases_not_assigned_count,...,staff_hired_ci_per_health_dept_mean,staff_hired_count,staff_hired_ct_count,staff_hired_ct_per_health_dept_mean,user_cases_notifying,users_with_code,cases_missed_count,cases_missed_perc,region,time_period
65,65,Digital Point Solutions for Extending Contact ...,2021-03-15,04/13/2021,state_a,21540.0,NaN,NaN,NaN,74869.0,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,T4
66,66,ENPA and ENCV metrics reported by States,2023-02-01,02/28/2023,CA,NaN,NaN,NaN,6751.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,T8
67,67,ENPA and ENCV metrics reported by States,2023-02-01,02/28/2023,DC,NaN,NaN,NaN,600.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,T8
68,68,ENPA and ENCV metrics reported by States,2023-02-01,02/28/2023,HI,NaN,NaN,NaN,107.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,T8
69,69,ENPA and ENCV metrics reported by States,2023-02-01,02/28/2023,MA,NaN,NaN,NaN,1354.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,T8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,241,NPR JHU colaborative survey,2021-12-01,01/01/2022,South Dakota,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,278.0,NaN,NaN,NaN,None,None,NaN,T6
242,242,NPR JHU colaborative survey,2021-12-01,01/01/2022,Tennessee,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1800.0,NaN,NaN,NaN,None,None,NaN,T6
243,243,NPR JHU colaborative survey,2021-12-01,01/01/2022,Texas,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3533.0,NaN,NaN,NaN,None,None,NaN,T6
244,244,NPR JHU colaborative survey,2021-12-01,01/01/2022,US Virgin Islands,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,13.0,NaN,NaN,NaN,None,None,NaN,T6


In [204]:
df['ratio_cases_contacts_intv_computed'] = df['cases_interviewed_count']/df['contacts_interviewed_count']
df['ratio_contacts_cases_intv_computed'] = df['contacts_interviewed_count']/df['cases_interviewed_count']
df['contacts_named_ratio_computed'] = df['contacts_named_count']/df['cases_interviewed_count']
df['contacts_named_naming_ratio_computed'] = df['contacts_named_count']/df['cases_named_contacts_count']
df['staff_per_assigned_ratio'] = (df['staff_hired_ci_count']+df['staff_hired_ct_count'])/(df['cases_assigned_count'] + df['contacts_named_count'])
df['case_assigned_per_staff_ratio'] = (df['cases_assigned_count'])/df['staff_hired_ci_count']
df['contact_assigned_per_staff_ratio'] = (df['contacts_named_count'])/df['staff_hired_ct_count']
df['ci_per_case_ratio'] = (df['staff_hired_ci_count'])/(df['cases_assigned_count'])
df['ct_per_contact_ratio'] = (df['staff_hired_ct_count'])/(df['contacts_named_count'])
df['case_per_ci_ratio'] = (df['cases_assigned_count'])/(df['staff_hired_ci_count'])
df['contact_per_ct_ratio'] = (df['contacts_named_count'])/(df['staff_hired_ct_count'])
df['cases_missed_count'] = df["cases_assigned_count"]-df["cases_reached_count"]
df['cases_missed_perc'] = (df["cases_missed_count"] / df["cases_assigned_count"]) * 100


In [205]:
df.round(2).to_csv('/Users/kaitlinsinger/wa_notify/DEN_future/data_extraction/manipulated_data/simplified_wide_df_with_date_loc_imputed.csv')